This notebook compares analytical and numerical evaluations of alchemical derivatives of a rotated and translated CH3OH molecule up to the third order in Hartree-Fock theory.  
The results are summerized in the last section.

In [1]:
from pyscf import gto,scf
import numpy as np
import pyscf
from scipy.spatial.transform import Rotation

pyscf.__version__

'2.2.1'

In [2]:
from aqa.FcMole import FcM_like

Rotating and translating CH3OH.

In [3]:
mol_coord = np.array([[-1.27549516,	0.03357710,	0.00000000],
                      [-2.10170657,	-0.87840808,	1.59107621],
                      [-2.10170657,	-0.87840808,	-1.59107621],
                      [-1.76544024,	1.99528228,	0.00000000],
                      [1.41253482,	-0.22423026,	0.00000000],
                      [2.19050133,	1.29874374,	0.00000000]])
# Coordinates in Angstrom
# C	-0.67496302	0.01776824	0.00000000
# H	-1.11217530	-0.46483357	0.84196133
# H	-1.11217530	-0.46483357	-0.84196133
# H	-0.93423081	1.05585799	0.00000000
# O	0.74748129	-0.11865755	0.00000000
# H	1.15916347	0.68726564	0.00000000
rotvec = np.array([1.2 * np.pi, 1.8 * np.pi, 0.8 * np.pi])
rot = Rotation.from_rotvec(rotvec)
mol_coord = rot.apply(mol_coord)
# print(mol_coord)
# print(np.linalg.norm(mol_coord[1]))
for i in range(6):
  mol_coord[i] += [22.4, 8.7, 11.8]
print(mol_coord)

[[21.51976504  8.14268562 12.53661552]
 [22.19523369  6.50972431 13.49736794]
 [19.9104428   7.49931626 11.51581996]
 [20.96614982  9.55062923 13.87808426]
 [23.39549112  9.16151172 10.8826461 ]
 [23.76170114 10.78607451 11.27170604]]


In [4]:
# target_mol = """
# C	-1.27549516	0.03357710	0.00000000
# H	-2.10170657	-0.87840808	1.59107621
# H	-2.10170657	-0.87840808	-1.59107621
# H	-1.76544024	1.99528228	0.00000000
# O	1.41253482	-0.22423026	0.00000000
# H	2.19050133	1.29874374	0.00000000
# """
target_mol = []
atom_symbols = ['C', 'H', 'H', 'H', 'O', 'H']
for i, atom_coords in enumerate(mol_coord):
    # print(atom_coords)
    atom_str = f' {atom_symbols[i]} {atom_coords[0]: .8f} {atom_coords[1]: .8f} {atom_coords[2]: .8f}'
    print(atom_str)
    target_mol.append(atom_str)
print(target_mol)
# dft_functional = "pbe0"  # "lda,vwn"
basis_set = "def2-TZVP"

 C  21.51976504  8.14268562  12.53661552
 H  22.19523369  6.50972431  13.49736794
 H  19.91044280  7.49931626  11.51581996
 H  20.96614982  9.55062923  13.87808426
 O  23.39549112  9.16151172  10.88264610
 H  23.76170114  10.78607451  11.27170604
[' C  21.51976504  8.14268562  12.53661552', ' H  22.19523369  6.50972431  13.49736794', ' H  19.91044280  7.49931626  11.51581996', ' H  20.96614982  9.55062923  13.87808426', ' O  23.39549112  9.16151172  10.88264610', ' H  23.76170114  10.78607451  11.27170604']


### Analytical evaluation

In [5]:
%load_ext autoreload
from aqa.AP_class import APDFT_perturbator as AP

In [6]:
mol_NN=gto.M(atom=target_mol,unit="Bohr",basis=basis_set)
mf_nn=scf.RHF(mol_NN)
# mf_nn.xc = dft_functional
mf_nn.scf()
# mf_nn.conv_tol = 1e-12
# mf_nn.grids.level = 6
# mf_nn.nuc_grad_method().kernel()
# ap_nn=AP(mf_nn,sites=[0,1,2,3,4,5])
ap_nn=AP(mf_nn,sites=[1,2,4,5])

converged SCF energy = -115.087746889062


In [7]:
analytical_1st_alchemical_derivative = ap_nn.build_gradient()
analytical_2nd_alchemical_derivative = ap_nn.build_hessian()
analytical_3rd_alchemical_derivative = ap_nn.build_cubic()

Inefficient alchemical force in HFT which calculates the response matrix for nuclear coordinates

In [8]:
# inefficient_analytical_alchemical_force = np.zeros((6, 6, 3))
# for i in range(6):
#   inefficient_analytical_alchemical_force[i] = ap_nn.build_inefficient_alchemical_force(i)

Efficient alchemical force in HFT which calculates the response matrix for nuclear coordinates

In [9]:
# efficient_analytical_alchemical_force = np.zeros((6, 6, 3))
# for i in range(6):
#   efficient_analytical_alchemical_force[i] = ap_nn.build_alchemical_force(i)

### Numerical evaluation

In [10]:
%autoreload 2
mol_NN=gto.M(atom=target_mol,unit="Bohr",basis=basis_set)
fc_param = 0.001
# fcs1 = [fc_param, 0.0, 0.0, 0.0, 0.0, 0.0]
# fcs2 = [-fc_param, 0.0, 0.0, 0.0, 0.0, 0.0]
fcs3 = [0.0, fc_param, 0.0, 0.0, 0.0, 0.0]
fcs4 = [0.0, -fc_param, 0.0, 0.0, 0.0, 0.0]
fcs5 = [0.0, 0.0, fc_param, 0.0, 0.0, 0.0]
fcs6 = [0.0, 0.0, -fc_param, 0.0, 0.0, 0.0]
# fcs7 = [0.0, 0.0, 0.0, fc_param, 0.0, 0.0]
# fcs8 = [0.0, 0.0, 0.0, -fc_param, 0.0, 0.0]
fcs9 = [0.0, 0.0, 0.0, 0.0, fc_param, 0.0]
fcs10 = [0.0, 0.0, 0.0, 0.0, -fc_param, 0.0]
fcs11 = [0.0, 0.0, 0.0, 0.0, 0.0, fc_param]
fcs12 = [0.0, 0.0, 0.0, 0.0, 0.0, -fc_param]

##### Efficient numerical alchemical force

In [11]:
# fmol1=FcM_like(mol_NN,fcs=fcs1)
# fmol2=FcM_like(mol_NN,fcs=fcs2)
fmol3=FcM_like(mol_NN,fcs=fcs3)
fmol4=FcM_like(mol_NN,fcs=fcs4)
fmol5=FcM_like(mol_NN,fcs=fcs5)
fmol6=FcM_like(mol_NN,fcs=fcs6)
# fmol7=FcM_like(mol_NN,fcs=fcs7)
# fmol8=FcM_like(mol_NN,fcs=fcs8)
fmol9=FcM_like(mol_NN,fcs=fcs9)
fmol10=FcM_like(mol_NN,fcs=fcs10)
fmol11=FcM_like(mol_NN,fcs=fcs11)
fmol12=FcM_like(mol_NN,fcs=fcs12)
mf=scf.RHF(mol_NN)
# mf1=scf.RKS(fmol1)
# mf2=scf.RKS(fmol2)
mf3=scf.RHF(fmol3)
mf4=scf.RHF(fmol4)
mf5=scf.RHF(fmol5)
mf6=scf.RHF(fmol6)
# mf7=scf.RKS(fmol7)
# mf8=scf.RKS(fmol8)
mf9=scf.RHF(fmol9)
mf10=scf.RHF(fmol10)
mf11=scf.RHF(fmol11)
mf12=scf.RHF(fmol12)
# mf.xc = dft_functional
# # mf1.xc = dft_functional
# # mf2.xc = dft_functional
# mf3.xc = dft_functional
# mf4.xc = dft_functional
# mf5.xc = dft_functional
# mf6.xc = dft_functional
# # mf7.xc = dft_functional
# # mf8.xc = dft_functional
# mf9.xc = dft_functional
# mf10.xc = dft_functional
# mf11.xc = dft_functional
# mf12.xc = dft_functional
# mf.conv_tol = 1e-12
# mf1.conv_tol = 1e-12
# mf2.conv_tol = 1e-12
# mf3.conv_tol = 1e-12
# mf4.conv_tol = 1e-12
# mf.grids.level = 6
# mf1.grids.level = 6
# mf2.grids.level = 6
# mf3.grids.level = 6
# mf4.grids.level = 6
# Without dm0=mf.init_guess_by_1e(), some SCFs do not converge.
e=mf.scf(dm0=mf.init_guess_by_1e())
# e1=mf1.scf(dm0=mf1.init_guess_by_1e())
# e2=mf2.scf(dm0=mf2.init_guess_by_1e())
e3=mf3.scf(dm0=mf3.init_guess_by_1e())
e4=mf4.scf(dm0=mf4.init_guess_by_1e())
e5=mf5.scf(dm0=mf5.init_guess_by_1e())
e6=mf6.scf(dm0=mf6.init_guess_by_1e())
# e7=mf7.scf(dm0=mf7.init_guess_by_1e())
# e8=mf8.scf(dm0=mf8.init_guess_by_1e())
e9=mf9.scf(dm0=mf9.init_guess_by_1e())
e10=mf10.scf(dm0=mf10.init_guess_by_1e())
e11=mf11.scf(dm0=mf11.init_guess_by_1e())
e12=mf12.scf(dm0=mf12.init_guess_by_1e())

converged SCF energy = -115.087746889061


/home/takafumi/program/pyscf/pyscf/gto/mole.py:1213: UserWarning: Function mol.dumps drops attribute with_rinv_at_nucleus because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -115.088877458771
converged SCF energy = -115.086617404048
converged SCF energy = -115.088877458771
converged SCF energy = -115.086617404047
converged SCF energy = -115.110098300524
converged SCF energy = -115.065399080753
converged SCF energy = -115.088765014498
converged SCF energy = -115.086729720902


In [12]:
# nuc_grad = mf.nuc_grad_method().kernel()
# # nuc_grad1 = mf1.nuc_grad_method().kernel()
# # nuc_grad2 = mf2.nuc_grad_method().kernel()
# nuc_grad3 = mf3.nuc_grad_method().kernel()
# nuc_grad4 = mf4.nuc_grad_method().kernel()
# nuc_grad5 = mf5.nuc_grad_method().kernel()
# nuc_grad6 = mf6.nuc_grad_method().kernel()
# # nuc_grad7 = mf7.nuc_grad_method().kernel()
# # nuc_grad8 = mf8.nuc_grad_method().kernel()
# nuc_grad9 = mf9.nuc_grad_method().kernel()
# nuc_grad10 = mf10.nuc_grad_method().kernel()
# nuc_grad11 = mf11.nuc_grad_method().kernel()
# nuc_grad12 = mf12.nuc_grad_method().kernel()

In [13]:
# efficient_numerical_alchemical_force = np.zeros((6, 6, 3))
# # efficient_numerical_alchemical_force[0] = (nuc_grad1 - nuc_grad2) / (2 * fc_param)
# efficient_numerical_alchemical_force[1] = (nuc_grad3 - nuc_grad4) / (2 * fc_param)
# efficient_numerical_alchemical_force[2] = (nuc_grad5 - nuc_grad6) / (2 * fc_param)
# # efficient_numerical_alchemical_force[3] = (nuc_grad7 - nuc_grad8) / (2 * fc_param)
# efficient_numerical_alchemical_force[4] = (nuc_grad9 - nuc_grad10) / (2 * fc_param)
# efficient_numerical_alchemical_force[5] = (nuc_grad11 - nuc_grad12) / (2 * fc_param)

#### Numerical alchemical derivatives

In [14]:
numerical_1st_alchemical_derivative = np.zeros((4))
# numerical_1st_alchemical_derivative[0] = (e1 - e2) / (2 * fc_param)
numerical_1st_alchemical_derivative[0] = (e3 - e4) / (2 * fc_param)
numerical_1st_alchemical_derivative[1] = (e5 - e6) / (2 * fc_param)
# numerical_1st_alchemical_derivative[3] = (e7 - e8) / (2 * fc_param)
numerical_1st_alchemical_derivative[2] = (e9 - e10) / (2 * fc_param)
numerical_1st_alchemical_derivative[3] = (e11 - e12) / (2 * fc_param)

#### Numerical alchemical hardness

In [15]:
ap_nn=AP(mf,sites=[1,2,4,5])
# ap_nn1=AP(mf1,sites=[0,1,2,3,4,5])
# ap_nn2=AP(mf2,sites=[0,1,2,3,4,5])
ap_nn3=AP(mf3,sites=[1,2,4,5])
ap_nn4=AP(mf4,sites=[1,2,4,5])
ap_nn5=AP(mf5,sites=[1,2,4,5])
ap_nn6=AP(mf6,sites=[1,2,4,5])
# ap_nn7=AP(mf7,sites=[0,1,2,3,4,5])
# ap_nn8=AP(mf8,sites=[0,1,2,3,4,5])
ap_nn9=AP(mf9,sites=[1,2,4,5])
ap_nn10=AP(mf10,sites=[1,2,4,5])
ap_nn11=AP(mf11,sites=[1,2,4,5])
ap_nn12=AP(mf12,sites=[1,2,4,5])
an = ap_nn.build_gradient()
# an1 = ap_nn1.build_gradient()
# an2 = ap_nn2.build_gradient()
an3 = ap_nn3.build_gradient()
an4 = ap_nn4.build_gradient()
an5 = ap_nn5.build_gradient()
an6 = ap_nn6.build_gradient()
# an7 = ap_nn7.build_gradient()
# an8 = ap_nn8.build_gradient()
an9 = ap_nn9.build_gradient()
an10 = ap_nn10.build_gradient()
an11 = ap_nn11.build_gradient()
an12 = ap_nn12.build_gradient()
ann = ap_nn.build_hessian()
# ann1 = ap_nn1.build_hessian()
# ann2 = ap_nn2.build_hessian()
ann3 = ap_nn3.build_hessian()
ann4 = ap_nn4.build_hessian()
ann5 = ap_nn5.build_hessian()
ann6 = ap_nn6.build_hessian()
# ann7 = ap_nn7.build_hessian()
# ann8 = ap_nn8.build_hessian()
ann9 = ap_nn9.build_hessian()
ann10 = ap_nn10.build_hessian()
ann11 = ap_nn11.build_hessian()
ann12 = ap_nn12.build_hessian()

In [16]:
numerical_2nd_alchemical_derivative = np.zeros((4, 4))
# numerical_2nd_alchemical_derivative[0] = (an1 - an2) / (2 * fc_param)
numerical_2nd_alchemical_derivative[0] = (an3 - an4) / (2 * fc_param)
numerical_2nd_alchemical_derivative[1] = (an5 - an6) / (2 * fc_param)
# numerical_2nd_alchemical_derivative[3] = (an7 - an8) / (2 * fc_param)
numerical_2nd_alchemical_derivative[2] = (an9 - an10) / (2 * fc_param)
numerical_2nd_alchemical_derivative[3] = (an11 - an12) / (2 * fc_param)

#### Numerical alchemical hardness derivatives

In [17]:
numerical_3rd_alchemical_derivative = np.zeros((4, 4, 4))
# numerical_3rd_alchemical_derivative[0, :, :] = (ann1 - ann2) / (2 * fc_param)
numerical_3rd_alchemical_derivative[0, :, :] = (ann3 - ann4) / (2 * fc_param)
numerical_3rd_alchemical_derivative[1, :, :] = (ann5 - ann6) / (2 * fc_param)
# numerical_3rd_alchemical_derivative[3, :, :] = (ann7 - ann8) / (2 * fc_param)
numerical_3rd_alchemical_derivative[2, :, :] = (ann9 - ann10) / (2 * fc_param)
numerical_3rd_alchemical_derivative[3, :, :] = (ann11 - ann12) / (2 * fc_param)

#### Inefficient numerical alchemical force

In [18]:
# mol_NN1=gto.M(atom="C 0 0 0.001; O 0 0 2.1",unit="Bohr",basis=basis_set)
# mol_NN2=gto.M(atom="C 0 0 -0.001; O 0 0 2.1",unit="Bohr",basis=basis_set)
# mol_NN3=gto.M(atom="C 0 0 0; O 0 0 2.101",unit="Bohr",basis=basis_set)
# mol_NN4=gto.M(atom="C 0 0 0; O 0 0 2.099",unit="Bohr",basis=basis_set)
# mf_nn1=scf.RKS(mol_NN1)
# mf_nn2=scf.RKS(mol_NN2)
# mf_nn3=scf.RKS(mol_NN3)
# mf_nn4=scf.RKS(mol_NN4)
# mf_nn1.xc = dft_functional
# mf_nn2.xc = dft_functional
# mf_nn3.xc = dft_functional
# mf_nn4.xc = dft_functional
# mf_nn1.scf()
# mf_nn2.scf()
# mf_nn3.scf()
# mf_nn4.scf()
# ap_nn1=AP(mf_nn1,sites=[0,1])
# ap_nn2=AP(mf_nn2,sites=[0,1])
# ap_nn3=AP(mf_nn3,sites=[0,1])
# ap_nn4=AP(mf_nn4,sites=[0,1])
# ad1 = ap_nn1.build_gradient()
# ad2 = ap_nn2.build_gradient()
# ad3 = ap_nn3.build_gradient()
# ad4 = ap_nn4.build_gradient()

In [19]:
# inefficient_numerical_alchemical_force = np.zeros((2, 2))  # only z axis components
# inefficient_numerical_alchemical_force[:, 0] = (ad1 - ad2) / (2 * fc_param)
# inefficient_numerical_alchemical_force[:, 1] = (ad3 - ad4) / (2 * fc_param)

## Comparison of analytical and numerical evaluations

$ \partial E/\partial Z_i $

In [20]:
print(analytical_1st_alchemical_derivative)

[ -1.13002724  -1.13002724 -22.3496097   -1.01764698]


$ (\partial E/\partial Z_i)_\mathrm{analytical} $ - $ (\partial E/\partial Z_i)_\mathrm{numerical} $

In [21]:
print(analytical_1st_alchemical_derivative - numerical_1st_alchemical_derivative)

[ 1.21000269e-07  1.21647476e-07  1.87589968e-07 -1.86182852e-07]


$\partial^2E/\partial Z_i\partial Z_j$

In [22]:
print(analytical_2nd_alchemical_derivative)

[[-1.08469606  0.32594434  0.28192602  0.26925656]
 [ 0.32594434 -1.08469606  0.28192602  0.26925656]
 [ 0.28192602  0.28192602 -3.60315557  0.44664312]
 [ 0.26925656  0.26925656  0.44664312 -0.95727943]]


$(\partial^2E/\partial Z_i\partial Z_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial Z_j)_\mathrm{numerical}$

In [23]:
print(analytical_2nd_alchemical_derivative - numerical_2nd_alchemical_derivative)

[[ 5.68423563e-07 -6.27566847e-07 -1.44508015e-07 -2.75701003e-07]
 [-6.27597489e-07  5.68435757e-07 -1.44478341e-07 -2.75700335e-07]
 [-2.83373867e-08 -2.83410193e-08 -6.46124310e-08 -8.10414404e-08]
 [-3.69954052e-07 -3.69957826e-07  1.59160196e-07 -4.98559349e-08]]


$\partial^3E/\partial Z_i\partial Z_j\partial Z_k$

In [24]:
print(analytical_3rd_alchemical_derivative)

[[[-0.85251327  0.07055889  0.05262468  0.04236894]
  [ 0.07055889  0.07055889  0.02811253  0.02466812]
  [ 0.05262468  0.02811253  0.02931532  0.02123612]
  [ 0.04236894  0.02466812  0.02123612  0.03413623]]

 [[ 0.07055889  0.07055889  0.02811253  0.02466812]
  [ 0.07055889 -0.85251328  0.05262468  0.04236894]
  [ 0.02811253  0.05262468  0.02931532  0.02123612]
  [ 0.02466812  0.04236894  0.02123612  0.03413623]]

 [[ 0.05262468  0.02811253  0.02931532  0.02123612]
  [ 0.02811253  0.05262468  0.02931532  0.02123612]
  [ 0.02931532  0.02931532 -0.17819706  0.09062195]
  [ 0.02123612  0.02123612  0.09062195  0.12409645]]

 [[ 0.04236894  0.02466812  0.02123612  0.03413623]
  [ 0.02466812  0.04236894  0.02123612  0.03413623]
  [ 0.02123612  0.02123612  0.09062195  0.12409645]
  [ 0.03413623  0.03413623  0.12409645 -0.81305105]]]


$(\partial^3E/\partial Z_i\partial Z_j\partial Z_k)_\mathrm{analytical}$ - $(\partial^3E/\partial Z_i\partial Z_j\partial Z_k)_\mathrm{numerical}$

In [25]:
print(analytical_3rd_alchemical_derivative - numerical_3rd_alchemical_derivative)

[[[ 1.03909669e-06  3.87181739e-08  1.70171582e-07  1.58467871e-07]
  [ 3.87181739e-08 -9.65259457e-07 -2.08809215e-07 -2.72506542e-07]
  [ 1.70171582e-07 -2.08809215e-07 -6.91901996e-08 -9.49163353e-08]
  [ 1.58467871e-07 -2.72506542e-07 -9.49163353e-08 -3.55932941e-07]]

 [[-9.65265417e-07  3.87176948e-08 -2.08808660e-07 -2.72506320e-07]
  [ 3.87176948e-08  1.03909625e-06  1.70170724e-07  1.58467022e-07]
  [-2.08808660e-07  1.70170724e-07 -6.91894818e-08 -9.49169533e-08]
  [-2.72506320e-07  1.58467022e-07 -9.49169533e-08 -3.55929763e-07]]

 [[ 2.70721612e-08  2.16981760e-08  1.11494243e-08  4.78478263e-08]
  [ 2.16981760e-08  2.70757433e-08  1.11500310e-08  4.78483463e-08]
  [ 1.11494243e-08  1.11500310e-08 -1.91810264e-07  2.87983533e-07]
  [ 4.78478263e-08  4.78483463e-08  2.87983533e-07  1.42428723e-07]]

 [[ 2.18962342e-07  2.12764920e-07  3.05793896e-08  5.87237849e-08]
  [ 2.12764920e-07  2.18970180e-07  3.05784386e-08  5.87232160e-08]
  [ 3.05793896e-08  3.05784386e-08  3.2023

$\partial^2E/\partial Z_i\partial R_j$

In [26]:
# print(efficient_analytical_alchemical_force)

$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{numerical}$

In [27]:
# print(efficient_analytical_alchemical_force - efficient_numerical_alchemical_force)

$\partial^2E/\partial Z_i\partial R_j$

In [28]:
# print(inefficient_analytical_alchemical_force)

$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{numerical}$

In [29]:
# print(inefficient_analytical_alchemical_force[:, :, 2] - inefficient_numerical_alchemical_force)

$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical,efficient}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical,inefficient}$

In [30]:
# print(efficient_analytical_alchemical_force - inefficient_analytical_alchemical_force)